In [1]:
# This program would read an excel file with the customers address and identification number then extracting the 
# city and zipcode from the line of address. Both zipcode and city is taken from the address line but the zipcode
# later on would be cross reference with the zipcode.xlsx file providing the zipcode with a matching province, 
# city/kabupaten, city/kabupaten name, district and village
#Program runtime ~ 255.78120374679565 seconds

import numpy as np
import pandas as pd
import re

In [2]:
# This would read the excel file that wants to be run through the program and is stored in data. Allzip would be
# the file to cross reference the zipcode in the address column with the zipcode of Indonesia. Then "ClientAddr"
# where the addresses are stored would be turn to all uppercase to prevent capitalization problem. Then in allzip,
# postalcode and index would be drop

data = pd.read_excel('Client Address Kevin.xlsx')
allzip = pd.read_excel('zipcode.xlsx')
data["ClientAddr"] = data["ClientAddr"].str.upper()
allzip = allzip.drop_duplicates(subset = 'Postal Code', keep = 'first').reset_index()
allzip = allzip.drop("index", axis = 1)

In [3]:
# In data, the blank spaces in ClientAddr are filled with 0, then replace all symbols like . and / with space so 
# splitting it word by would would be easy. Remane postal code with Zip for future merge. For loop would go through
# the divided address and check if a zipcode is present, and if it is, then it would be stored in lst. 

data['ClientAddr'] = data['ClientAddr'].fillna("0")
data['clean1'] = data['ClientAddr'].str.replace("."," ").replace('/',' ').str.split(" ")
data['clean2'] = data['ClientNumber']
allzip = allzip.rename(columns = {'Postal Code':'Zip'})
lst = []
index = 0

for addr in data['clean1']:
    for char in addr:
        if len(char) == 5 and char.isdigit() == True:
            lst.append([char,str(index)])
    index +=1

In [4]:
# Reset index would bring up the column index, and changing the column 'index' turning it to string. lst is stored
# in a dataframe and header name are changed for future merge. Then duplicates for both index and zip are drop and 
# data and df are merge while filling the cases of address with no zipcode with 00000

data = data.reset_index()
data['index'] = data['index'].astype(str)
df = pd.DataFrame(lst)
df.columns = ['Zip', 'index']
df = df.drop_duplicates(subset = ['Zip', 'index'], keep = 'last')
df = df.drop_duplicates(subset = 'index', keep = 'last')
df_temp = pd.merge(data, df, on = 'index', how = 'left')
df_temp['Zip'] = df_temp['Zip'].fillna("00000")

In [5]:
# df_temp now would be compared towards the list 'citylist' to check if the city in city list is located in df_temp
# ClientAddr, if it is then it would add the city name towards the database. Database would drop index, clean1 and
# clean2

df_temp['kota'] = ""
citylist = ["MALANG", "SAMARINDA", "TASIKMALAYA", "PONTIANAK", "BANJAMASIN", "SERANG",
       "JAMBI", "BALIKPAPAN", "SURAKARTA", "CIMAHI", "MANADO", "KUPANG", "JAYAPURA",
       "MATARAM", "YOGYAKARTA", "CILEGON", "AMBON", "BENGKULU", "PALU", "KENDARI", "SUKABUMI",
       "CIREBON", "PEKALONGAN", "KEDIRI", "PEMATANGSIANTAR", "TEGAL", "SORONG", "BINJAI", "DUMAI",
       "PALANGKA RAYA", "SINGKAWANG", "PROBOLINGGO", "BITUNG", "BANJARBARU", "TERNATE", "LUBUKLINGGAU",
       "PASURUAN", "TANJUNG PINANG", "PANGKAL PINANG", "MADIUN", "TARAKAN", "BATU", "GORONTALO",
       "BANJAR", "LHOKSEUMAWE", "PRABUMULIH", "PALOPO", "LANGSA", "SALATIGA", "PAREPARE", 
       "TEBING TINGGI", "TANJUNGBALAI", "METRO", "BOTANG", "BAUBAU", "BLITAR", "GUNUNGSITOLI",
       "BIMA", "PAGAR ALAM", "MOJOKERTO", "PAYAKUMBUH", "MAGELANG", "KOTAMOBAGU", "BUKITTINGGI",
       "TIDORE", "SUNGAI PENUH", "TOMOHON", "SIBOLGA", "PARIAMAN", "TUAL", "SUBULUSSALAM", "SOLOK",
       "SAWAHLUNTO", "PADANG PANJANG", "SABANG", "MALUKU", "JEMBER", "PAPUA", "KLATEN",
       "PARINDU SANGGAU", "BINTAN", "RIAU", "BANTEN", "DEPOK", "CIAMIS", "KARIMUN", "JAKARTA", "SURABAYA", 
       "BANDUNG", "MEDAN", "SEMARANG", "PALEM BANG", "ACEH"
       "MAKASSAR", "BATAM", "PEKANBARU", "BANDAR LAMPUNG", "LAMPUNG", "BALI", "DENPASAR",
       "BEKASI", "DEPOK", "TANGERANG", "BATAM", "BOGOR", "PADANG", "PEKANBARU",]

for i in range(len(citylist)):
    df_temp['kota'] = np.where(df_temp['ClientAddr'].str.contains(citylist[i]),citylist[i],df_temp['kota'])
df_temp['kota'] = np.where(df_temp['ClientAddr'].str.contains("SUROBOYO"),"SURABAYA",df_temp['kota'])
df_temp['kota'] = np.where(df_temp['ClientAddr'].str.contains("DENPASAR"),"BALI",df_temp['kota'])
df_temp = df_temp.drop(['index','clean1','clean2'], axis = 1)

In [7]:
# Turning 'Zip' for both df_temp and allzip to string type for easier merge. Then it would all be merge to df_temp
# by the zip. 'No' column in df_temp is now dropped.

allzip['Zip'] = allzip['Zip'].astype(str)
df_temp['Zip'] = df_temp['Zip'].astype(str)
df_temp = pd.merge(df_temp, allzip, on = 'Zip', how = 'left')
df_temp = df_temp.drop(['No.'], axis = 1)

In [9]:
# This would write the dataframe towards an excel file titled "Output" 

with pd.ExcelWriter ('Output.xlsx') as writer:
    df_temp.to_excel (writer, sheet_name = "Address", index = False)
writer.save()

255.78120374679565
